In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import numpy as np
from pymystem3 import Mystem
import re

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

import itertools
import warnings
warnings.filterwarnings('ignore')
import pickle

Посмотрим на тестовые данные, которые предоставил заказчик

In [5]:
soup = BeautifulSoup()
with open('test.csv', 'r') as f:
    test = f.read()
    soup = BeautifulSoup(test, 'lxml')
    test_reviews = soup.findAll('review')
    data_test = []
    for review in test_reviews:
        data_test.append(review.text)

In [6]:
data_test = pd.DataFrame(data_test)

In [7]:
data_test.head()

,0
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях\nК сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


In [8]:
data_test.shape

(100, 1)

In [9]:
data_test[0][1]

'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если \nпостоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.\n Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсорниках и кнопочных -просто издивательствогромкий ,чистый звук (хорошо варьируется как в + так и в -)значение hot кнопок (верхняя панель до основной раскладки цифры/буквы) задается под себямного цветных панелек с пластиковым защитным  экраном,переставляются легко(те родной экран телефона никогда не поцарапается)кнопки не стираютсякак не странно достойные фото, нет не спорю не 25 мегапикселей, но  отснять рассписание или конспекты, зафотать пейзаж за окном автобуса получается вполне пристойносохранение файлов,отснятых фото, переброшенных песен происходит  на карту памяти  и это оч удобно, карточки кушает до 32 Гб !удобный ашевский бонус смс чат с аббанентом\nт.е.  вы может

### Поиск данных для обучающей выборки

Попробуем взять данные с goods.ru. Нам нужны отзывы на мобильные телефоны.

In [10]:
rev = []
for i in range(1,93):
    t_sleep = np.random.randint(1, 100)*0.1
    time.sleep(t_sleep)
    url = 'https://goods.ru/review/smartfony/page-'+str(i)
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    reviews = soup.findAll('div', {'class':'review'})
    for review in reviews:
        content = review.findAll('div', {'class':'sp-review-content'})
        rating_find = review.find('div', {'class': 'sp-review-ratings-common'}).text
        rating_list = rating_find.split()
        rating = int(rating_list[2])
        for parts in content:
            tmp = []
            text = review.findAll('div', {'class':['sp-review-pros-content sp-review-text-content', \
                                                   'sp-review-pros-label sp-review-text-label']})
            
            
            for item in text:
                tmp.append(item.text)
                
                

        rev.append((tmp, rating))


In [12]:
pos = []
neg = []
comment = []
for r in rev:
    rating = r[1]
    try:
        pos_idx = r[0].index('Достоинства')
        pos.append(r[0][pos_idx + 1])
    except:
        pass
    
    try:
        neg_idx = r[0].index('Недостатки')
        neg.append(r[0][neg_idx + 1])
    except:
        pass
    
    try:
        comment_idx = r[0].index('Комментарий')
        comment = r[0][comment_idx + 1]
        if rating >= 4:
            pos.append(comment)
        elif rating < 3:
            neg.append(comment)
        
    except:
        pass


In [13]:
len(pos)

2089

In [14]:
len(neg)

863

In [15]:
rev = open('train_data.txt', 'w')
rev.close()

In [41]:
neg.append('Очень плохой телефон')
neg.append('телефон просто отвратителен')


### Токенизация и лемматизация

In [42]:
from pymystem3 import Mystem
import re

In [43]:
def lemmatize(texts):
    tmp_lemmas = []
    lemmas = []
    m = Mystem()
    for text in texts:
        lem = []
        clean_text = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', text)
        tkn = clean_text.split()
        for t in tkn:
            lem.append(m.lemmatize(t))
        tmp_lemmas.append(lem)
    
    for sent in tmp_lemmas:
        if len(sent) > 2:
            tmp = []
            for word in sent:
                tmp.append(word[0].rstrip('\n'))
            lemmas.append(' '.join(tmp))

    return lemmas

In [44]:
with open('lemmatization.pkl', 'wb') as f:
    pickle.dump(lemmatize, f)

In [45]:
neg_lemmas = lemmatize(neg)

In [46]:
neg_lemmas[0]

'тормозной прошивка перегрев при активный использование не игра'

In [47]:
pos_lemmas = lemmatize(pos)

In [48]:
len(neg_lemmas), len(pos_lemmas)

(639, 1944)

In [49]:
data = neg_lemmas + pos_lemmas
labels = [0] * len(neg_lemmas) + [1] * len(pos_lemmas)

In [50]:
df = pd.DataFrame({'reviews': data, 'label': labels})

In [51]:
df

,reviews,label
0,тормозной прошивка перегрев при активный испол...,0
1,не быть в комплект штучка чтобы открывать отсе...,0
2,разве что аккумулятор быстро разрезаться даже ...,0
3,динамик немного искажать голос собеседник при ...,0
4,пока не находить,0
...,...,...
2578,стильный дизайн компактность легко ложиться в ...,1
2579,покупать в эльдорадо зима 2016 работать хорошо...,1
2580,2 симка для работа пойд т,1
2581,камера отличный батарея держаться 6 7 час а ес...,1


### Обучение моделей

In [52]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [53]:
for vect, clf in itertools.product([CountVectorizer, TfidfVectorizer],[LogisticRegression, LinearSVC, SGDClassifier]):
    print(vect, clf)
    print(cross_val_score(text_classifier(vect(), clf()), data, labels).mean())
    print("\n")

<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8714672861014324


<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.svm.classes.LinearSVC'>
0.8497870692992645


<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.856755710414247


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.linear_model.logistic.LogisticRegression'>
0.8346883468834688


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.svm.classes.LinearSVC'>
0.873015873015873


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'>
0.8610143244289586




In [54]:
clf = Pipeline(
            [("vectorizer", TfidfVectorizer()),
            ("classifier", SGDClassifier(random_state = 777))]
        )


clf.fit(lemmatize(data), labels)


Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                ('classifier',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='opti

In [55]:
with open('sentiment_review_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [56]:
test_lemmas = lemmatize(data_test[0])

In [57]:
y_pred = clf.predict(test_lemmas)

In [58]:
y_pred

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0])

In [59]:
data_test['y'] = y_pred


In [60]:
preds = []
for y in y_pred:
    if y == 0:
        preds.append('neg')
    elif y == 1:
        preds.append('pos')

In [61]:
submission = pd.DataFrame({'Id': range(0,100),'y': preds})
submission

,Id,y
0,0,neg
1,1,pos
2,2,neg
3,3,pos
4,4,pos
...,...,...
95,95,neg
96,96,pos
97,97,neg
98,98,pos


In [62]:
with open('result.csv', 'w') as f:
    f.write(submission.to_csv(sep=',', index=False))

In [63]:
import joblib
classifier = joblib.load('sentiment_review_classifier.pkl')

In [64]:
rev1 = data_test[0][1]
rev1

'ценанадежность-неубиваемостьдолго держит батарею 4 дня стабильно как телефон, 3-4 как плеер если \nпостоянно долбиться в уши и звонить по паре часо на дню, игры и, конечно,  смс , в месяц около 200 шт набирается.\n Максимальное время работы 5 дней в щадящем режиме.2 simqwerty рулит -после нее набор смс на обычных сенсорниках и кнопочных -просто издивательствогромкий ,чистый звук (хорошо варьируется как в + так и в -)значение hot кнопок (верхняя панель до основной раскладки цифры/буквы) задается под себямного цветных панелек с пластиковым защитным  экраном,переставляются легко(те родной экран телефона никогда не поцарапается)кнопки не стираютсякак не странно достойные фото, нет не спорю не 25 мегапикселей, но  отснять рассписание или конспекты, зафотать пейзаж за окном автобуса получается вполне пристойносохранение файлов,отснятых фото, переброшенных песен происходит  на карту памяти  и это оч удобно, карточки кушает до 32 Гб !удобный ашевский бонус смс чат с аббанентом\nт.е.  вы может

In [65]:
r = lemmatize([rev1])
r

['ценанадежность неубиваемостьдолго держать батарея 4 день стабильно как телефон 3 4 как плеер если постоянно долбиться в ухо и звонить по пара часо на день игра и конечно смс в месяц около 200 шт набираться максимальный время работа 5 день в щадить режим 2 simqwerty рулить после она набор смс на обычный сенсорник и кнопочный просто издивательствогромкий чистый звук хорошо варьироваться как в так и в значение hot кнопка верхний панель до основной раскладка цифра буква задаваться под себямный цветной панелька с пластиков защитный экран переставляться легко тот родной экран телефон никогда не поцарапаться кнопка не стираютсякак не странно достойный фото нет не спорить не 25 мегапиксел но отснимать рассписание или конспект зафотать пейзаж за окно автобус получаться вполне пристойносохранение файл отснимать фото перебрасывать песня происходить на карта память и это оч удобно карточка кушать до 32 гб удобный ашевский бонус смс чат с аббанент т е вы мочь видеть весь ветка беседа с конкретный

In [66]:
pred = classifier.predict([' очень плохой телефон'])[0]
print('Предсказанная тональность:', pred)

Предсказанная тональность: 0
